## Opis projekta

Na web forumu bitcointalk.org pod sekcijom "Bitcoin Discussion" se svakodnevno objavljuju teme za raspravu od strane
raznih korisnika istog foruma. Pretpostavka je da će broj objavljenih tema na dnvenoj bazi kao i da će rezultati analize sentimenata teksta tih tema dati povratnu informaciju o javnom interesu za Bitcoin kriptovalutom. Tekst tema kao i datum objave teme iz navedenog foruma ostrugati (eng.scrape) će se pomoću BeautfulSoup i requests biblioteka.
Analiza sentimenta teksta objavljenih tema vršiti će se upotrebom Transformes bilbioteke, točinije njezinog osnovnog deep learning modela za klasifikaciju teksta naziva "sentiment-analysis".
Rezultati analize sentimenta teksta poslužiti će kao dio ulaznog seta podataka s pretpostavkom da će poboljšati uspješnost modela u predviđanju rasta ili pada cijene Bitcoina na dnevnoj bazi.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import re

subject=[]
dat=[]
for i in range(0,25000,40):
    # svaka stranica ima url pomaknut za 40 gledajući znak poslje ...board=1.
    # 1. str ima ...board=1.0, 2. str....board=1.40 ...
    url = f'https://bitcointalk.org/index.php?board=1.{i}'
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    
    date_element= soup.find_all(class_='windowbg2 lastpostcol')
    posts = soup.find_all(class_='windowbg') 
    # find_all funkcija vraća listu elemenata iz kojih se izvlači datum 
    for date in date_element:
        # 2 elementa liste spremljna u jednu str varijablu
        date=str(list(date.descendants)[5:7])
        # ako element ima više od 20 znakova sadrži poni datum i vrijeme
        if len(date)>20:
            date=date[20:]
            # search funkcija za pronalazak datuma (npr. February 22, 2023)
            # REGEX:
            # veliko 1.slovo: [A-Z]
            # 2-8 malih slova: [a-z]{2,8}
            # \s:  razmak(whitespace)
            # dan:  \d{2}
            # zarez: \,
            # godina: \d{4}          
            date=re.search('^[A-Z][a-z]{2,8}\s\d{2}\,\s\d{4}',date).group(0)
            #datum sadrži zarez(,) potrebno ga je ukloniti sub funkcijom
            date=re.sub("\,", "", date)
        # ako ne, radi se o oznaci današnjeg datuma //n...<b>Today<b>
        else:
            #pri čemu se koristi datetime bibliteka za definiranje
            # današnjeg datuma u istom obliku 
            date=datetime.datetime.now().strftime("%B %d %Y")
        dat.append(date)
    # prolaz kroz svaki element klase 'windowbg'
    for post in posts:
        text=post.find('a')
        # pošto su neki elementi za izlaz daju "None" vrijednost
        try:
            text=text.string
        # ako je text=None samo nastavi
        except:
            continue
        # dodaj text varijablu u subject listu 
        subject.append(text)
        
# dat i subject liste se pretvarju u Series stupce
# zbog potrebe spajanja u tablicu naredbom concat
dat=pd.Series(dat)
# 1. stranica daje jedan element None viška, pa se počinje od 2. reda
# kako bi bili datumi usklađeni s temama koje su objavljene na taj datum
subject=subject[1:]
subject=pd.Series(subject)

df=pd.concat({"Date": dat ,"subject": subject}, axis=1)
df=df.iloc[3:]

In [ ]:
# provjera jesu li svi datumi točno ispisani
for i in range(2300,500,-150):
    print(df.Date[i])

In [6]:
import time
# od 3 jer tablica počinje od 3 indexa do len +3 
for i in range(3,len(df.Date)+3):
    # najprije u time.struct_time touple format
    df.Date[i]=time.strptime(df.Date[i],"%B %d %Y")
    # pa u %Y-%m-%d format
    df.Date[i]=time.strftime("%Y-%m-%d",df.Date[i])

In [7]:
df=df.set_index('Date')

In [8]:
df=df.iloc[::-1]

In [9]:
# potrebno je opet zadati brojevne indexe jer ako su
#datumi postavljeni za index stupac ne prepoznaju se 
# kao zasebni stupac
df=df.reset_index()

In [79]:
# pretvaranje dobivene tablice u listu
df_list = df.to_dict('records')

#### postavljanje datuma kao glavnog ključa koji sadrži listu tema za taj dan

In [ ]:
teme={}
for dic in df_list:
    datum=dic["Date"]
    if datum not in teme:
        teme[datum]=dict(teme_list=list())
    tema=dic["subject"]
    teme[datum]["teme_list"].append(tema)

In [89]:
#teme

### Uvoz bilblioteke Transformers
* modul `pipeline` omogućuje upotrebu osnovnog deep learning modela za analizu teksta
* deep learning model `sentiment-analysis` analizira tekst i vraća rezulte u brojevnom obliku
* ključevi dict-a su `label` koji za vrijednost ima polaritet (Positive/Negative) i score koji označuje rezultat od 0-1 i 

In [91]:
from transformers import pipeline
sent_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


#### Izrada funkcije za analizu komentara
* Ako je uneseni tekst unutar modela pozitvno orijetitan poput "like", "great", "rise", `label` će biti `POSITIVE`
* Tekst poput "hate", "collapse", "fallout" će vratit vrijednost label ključa `NEGATIVE`  
* Na osnovu tih vrijednosti se izrađuje funkcija koja ako je label NEGATIVE vraća negativnu vrijednost rezultata
* analiziranje teksta staje na 100 znakova unesenog teksta (`tekst[:100]`) kako bi se funkcija izvršila u što kraćem roku, što ubrzava izvođenje daljnjeg programskog koda 
* index [0] se postavlja jer je sentiment_piline vraća dictonary unutar liste, 0 je prvi element liste

In [ ]:
def analiza_sent(tekst):
    tema=sent_pipeline(tekst[:100])[0]
    rezultat=tema["score"]
    polaritet=tema["label"]
    if polaritet=="NEGATIVE":
        rezultat*=-1
    return rezultat

In [ ]:
rez_transf={}
for dat in teme:
    lista_tema = teme[dat]["teme_list"]
    rez_transf[dat]=dict(broj_tema=len(lista_tema), rezultati=list())
    for tema in lista_tema:
        rez_transf[dat]["rezultati"].append(analiza_sent(tema))

In [ ]:
# funkcije za odvajanje pozitvinih od negativnih komentara
def pozitivni(lista):
    poz=[]
    for br in lista:
        if br >= 0:
            poz.append(br)
    return poz

def negativni(lista):
    neg=[]
    for br in lista:
        if br < 0:
            neg.append(br)
    return neg

In [96]:
end_transf={}

for k in rez_transf:
    rez=rez_transf[k]["rezultati"]
    end_transf[k]=dict(broj_tema=rez_transf[k]["broj_tema"],avg_rezultata=float,
                       posto_poz=float,posto_neg=float)
    end_k=end_transf[k]
    end_k["avg_rezultata"]=sum(rez)/len(rez) 
    end_k["posto_poz"]=len(pozitivni(rez))/len(rez)
    end_k["posto_neg"]=len(negativni(rez))/len(rez)

In [97]:
#end_transf

## Prebacivanje rezultata u pandas df

In [ ]:
# pretvarnje end_transf dictionary-a u tablicu
df_transf=pd.DataFrame.from_dict(end_transf,orient="index")

In [103]:
# Postavljanje datetime index-a
df_transf.index=pd.to_datetime(df_transf.index)

In [104]:
# definiranje varijble koja će postati novi index
# postavljnje početnog i završnog datuma
datumi=pd.date_range(start="10-09-2017",end="01-23-2023")

In [105]:
# reindeksirenje i ispunjavanje nepostojećih datuma s vrijednostima 0
df_reindex=df_transf.reindex(datumi,fill_value=0)

In [1]:
# provjera ineksa koji su dodani pri reindeksiranju
null=[]
for i in range(len(df_reindex)):
    if df_reindex["broj_kom"].iloc[i]==0:
        null.append(i)

print(f"Index vrijednosti nula = {null}")

In [ ]:
# upotreba rolling funkcije za pronalazak srednje vrijednosti posljednjih 7 dana
rolling_transf=df_reindex.rolling(7).mean()

In [205]:
# izbacivanje NaN vrijednosti koje su kreirane rolling funkcijom
rolling_transf=rolling_transf.dropna()

In [205]:
# Prebacivanje konačne tablice u .csv dokument
rolling_transf.to_csv("Transf_roll7.csv")

### Upotreba funkcije rolling
* rolling omogućje izračun srednje vrijednosti podataka u posljednjih 7 dana srednja vrijednost od 1. do 7. reda zapisan je u 7. red, 8 red ima zapisanu srednju vrijednost od 2. do 8. reda i tako do kraj tablice
* prvih 7 redova ima NaN vrijednosti je nemaju prethodnih podataka za izračun pa ih je potrebno ukloniti